In [1]:
import logging

# Cria um logger com o nome 'jupyter_logger'
logger = logging.getLogger('jupyter_logger')

# Define o nível de log para DEBUG, então todas as mensagens de log serão mostradas
logger.setLevel(logging.DEBUG)

# Cria um manipulador de log que escreve as mensagens de log na saída padrão
handler = logging.StreamHandler()

# Define o nível de log do manipulador para DEBUG
handler.setLevel(logging.DEBUG)

# Cria um formatador de log que adiciona a data e hora à mensagem de log
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Adiciona o formatador ao manipulador
handler.setFormatter(formatter)

# Adiciona o manipulador ao logger
logger.addHandler(handler)

# Agora você pode usar o logger para registrar mensagens de log
logger.debug('Esta é uma mensagem de debug')
logger.info('Esta é uma mensagem de info')
logger.warning('Esta é uma mensagem de warning')
logger.error('Esta é uma mensagem de error')
logger.critical('Esta é uma mensagem de critical')

2024-05-20 01:11:43,464 - jupyter_logger - DEBUG - Esta é uma mensagem de debug
2024-05-20 01:11:43,466 - jupyter_logger - INFO - Esta é uma mensagem de info
2024-05-20 01:11:43,468 - jupyter_logger - WARNING - Esta é uma mensagem de warning
2024-05-20 01:11:43,469 - jupyter_logger - ERROR - Esta é uma mensagem de error
2024-05-20 01:11:43,471 - jupyter_logger - CRITICAL - Esta é uma mensagem de critical


In [2]:
from confluent_kafka import Producer
# topic = "user"

class Kafka_producer:
    def __init__(self,broker = "kafka-broker:9092",group_id = "python"):
        self._broker = broker
        self._group_id = group_id
        self._producer = self._making_the_producer()
        return
    def _making_the_producer(self):
        logger.debug('Configuring code')
        config = {
                'bootstrap.servers': self._broker,
                'group.id': self._group_id,
                'auto.offset.reset': 'latest',
                'enable.auto.commit': 'false',
                'max.poll.interval.ms': '86400000'
            }
        return Producer(config)
    
    def kafka_producer(self,topic, data):

        def _delivery_report(err, msg):
            if err is not None:
                logger.debug('Message delivery failed: {}'.format(err))
            else:
                logger.debug('Message delivered to {} [{}]'.format(msg.topic(), msg.partition()))
        logger.debug('Publishing to topic')
        self._producer.produce(topic, data, callback=_delivery_report)
        self._producer.flush()

In [3]:
import requests

def get_data_from_endpoint():
    url = "https://random-data-api.com/api/v2/users?size=1"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

In [ ]:
import json
from time import sleep


producer = Kafka_producer()

while True:
    logger.debug('sending mensage')
    producer.kafka_producer(topic="user", data=json.dumps(get_data_from_endpoint()))
    sleep(10)

2024-05-20 01:11:45,090 - jupyter_logger - DEBUG - Configuring code
2024-05-20 01:11:45,100 - jupyter_logger - DEBUG - sending mensage
2024-05-20 01:11:45,715 - jupyter_logger - DEBUG - Publishing to topic
2024-05-20 01:11:46,183 - jupyter_logger - DEBUG - Message delivered to user [0]
2024-05-20 01:11:56,194 - jupyter_logger - DEBUG - sending mensage
2024-05-20 01:11:56,932 - jupyter_logger - DEBUG - Publishing to topic
2024-05-20 01:11:56,939 - jupyter_logger - DEBUG - Message delivered to user [0]
2024-05-20 01:12:06,945 - jupyter_logger - DEBUG - sending mensage
2024-05-20 01:12:07,779 - jupyter_logger - DEBUG - Publishing to topic
2024-05-20 01:12:07,785 - jupyter_logger - DEBUG - Message delivered to user [0]
2024-05-20 01:12:17,797 - jupyter_logger - DEBUG - sending mensage
2024-05-20 01:12:18,408 - jupyter_logger - DEBUG - Publishing to topic
2024-05-20 01:12:18,413 - jupyter_logger - DEBUG - Message delivered to user [0]
2024-05-20 01:12:28,425 - jupyter_logger - DEBUG - sendi